# Assignment 2
## Student name: Wenzhuo Li
## Stundent Number: 217101841

**Task 1**: We should import the necessary libraries and link with database

In [3]:
from ucimlrepo import fetch_ucirepo

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

Load the X and y from the adult.data.features and adult.data.targets respectively.

In [4]:
adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets
# metadata
print(adult.metadata)
# variable information
print(adult.variables)

ConnectionError: Error connecting to server

**Task 2** :Take a quick look at the data structure (i.e., X) using .head(), .info(), .describe(), and .shape.

In [ ]:
# .head
X.head()

In [ ]:
# .info
X.info()

In [ ]:
# .describe
X.describe()

In [ ]:
# .shape
X.shape

**Task 2.1:** Plot a histogram of the data.

In [ ]:
X.hist(figsize=(24, 16))
plt.show()

**Task 3:** There are missing values in this dataset that are entered as ?, check for the number of these missing values.

In [ ]:
(X == '?').sum()

**Task 4:** Replace the missing values you found in the previous step with null (nan). Run a X.info() to see the non-null count.

In [ ]:
X_Features = X.copy()
X_Features.replace('?', np.NaN, inplace=True)
X.info()

**Task 5:** Create and apply a preprocessing pipeline to:
1. Fill in the missing numerical values with the mean using a SimpleImputer.
2. Scale the numerical columns using StandardScaler. Do not scale the target.
3. Fill in the missing categorical values with the most_frequent value using SimpleImputer.
4. Encode the categorical columns using OneHotEncoder. Do not encode the target.
- Display your pipeline.
- Print X_prepared.shape.

In [ ]:
# For extract the columns of number
num_col = X_Features.select_dtypes(include='number').columns.to_list()
# For extract the columns of non-number
cat_col = X_Features.select_dtypes(exclude='number').columns.to_list()
#make a pipeline
num_pipline = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
cat_pipline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(sparse_output=False))
preprocessing = ColumnTransformer([('num', num_pipline, num_col),
                                       ('cat', cat_pipline, cat_col)],
                                      remainder='passthrough')
adult_prepared = preprocessing.fit_transform(X_Features)
feature_names = preprocessing.get_feature_names_out()
X_prepared = pd.DataFrame(data=adult_prepared, columns=feature_names)
X_prepared.shape

**Task 6:** Check the target value_counts. You will notice that the target needs some data cleaning.

In [ ]:
value_counts = y.value_counts()
value_counts

**Task 7:** Remove the period at the end of the >50K. and <=50K. i.e., replace all instances that are <=50K. with <=50K , and replace all the instances that are >50K. with >50K

In [ ]:
Y_Target = y.copy()
Y_Target.replace('>50K.', '>50K',inplace=True)
Y_Target.replace('<=50K.', '<=50K', inplace=True)
value_counts=Y_Target.value_counts()
value_counts

**Task 8:** Split the data into 80% training set and 20% testing set, print the shape of X_train, X_test, y_train, y_test in one command.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, Y_Target, test_size=0.2,random_state=42)
X_train.shape, y_train.shape,X_test.shape,y_test.shape

# Task 9:
 Train a svm model (svc) to predict if the income of the adult exceeds 50K on the training set using: kernel = poly, gamma = 1, and C =0.1. Call your model model_svm.

In [ ]:
model_svm = svm.SVC(kernel='poly',gamma=1,C=0.1)
model_svm.fit(X_train.iloc[:10000],y_train.iloc[:10000].values.ravel())
y_predict =  model_svm.predict(X_test)

**Task 9.1:** Test your model on the X_Test, and report the classification_report on the y_test and y_predict.

In [ ]:
print(classification_report(y_test,y_predict))

**Task 9.2:** Display the confusion matrix of your test results using

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_predict)

# Task 10:
 Use GridSearchCV to find the best value of kernel, gamma, and C.

**Task 10.1:** Split the dataset into 60% training, 20% validation, and 20% testing. Use the code below to perform the split:

In [ ]:
X_train,X_validation_test,y_train,_y_validation_test = train_test_split(X_prepared,y,test_size=0.4,random_state=42)
X_validation,X_test,y_validation,y_test = train_test_split(X_validation_test,_y_validation_test,test_size=0.5,random_state=42)
print(X_train.shape,y_train.shape,X_validation.shape,y_validation.shape,X_test.shape,y_test.shape)

**Task 10.2:** Use the below code snippet to pass the following hyperparameters for the GridSearchCV to find the best ones:

In [ ]:
svm_parameters = {
    'kernel':['rbf'],
    'C':[0.01,0.1,1,10],
    'gamma':[0.01,1,10]
}
svm = svm.SVC()
svm_gs = GridSearchCV(estimator=svm,
                      param_grid=svm_parameters)
svm_gs.fit(X_train.iloc[:10000],y_train.iloc[:10000].values.ravel())

svm_winner = svm_gs.best_estimator_
print('The best parameters are : ')
print(svm_winner)
print("The score is: ")
print(svm_winner.score(X_validation,y_validation))